# Immersive exploratory analysis

**Activities**

* Load and process Immersive Survey data
* Merge and output



## Preliminaries

Such as paths

In [33]:
%matplotlib inline
#NB I install a standard set of directories

#Get the top path
top_path = os.path.dirname(os.getcwd())

#Create the path for external data
ext_data = os.path.join(top_path,'data/external')

proc_data = os.path.join(top_path,'data/processed')

fig_path = os.path.join(top_path,'reports/figures')

## Load data

### Metadata

In [34]:
#Load nspl to get TTWAs and GORS
nspl = pd.read_csv(ext_data+'/NSPL_AUG_2017_UK.csv')

#Load the metadata
ttwa_names = pd.read_table(ext_data+'/ttwa_names_codes.txt',delimiter='\t')
gor_names = pd.read_table(ext_data+'/Region names and codes EN as at 12_10 (GOR).txt',delimiter='\t')


#Create name lookups
ttwa_codes_names_lookup = {x:y for x,y in zip(ttwa_names['TTWA11CD'],
                                             ttwa_names['TTWA11NM'])}

gor_names_lookup = {x:y if 'pseudo' not in y else y[9:] for x,y in zip(gor_names['GOR10CD'],gor_names['GOR10NM'])}

/usr/local/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (30,35) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [35]:
# Survey data dictionary
#Load
data_dict = pd.read_excel(ext_data+'/11_02_2018_survey_data_dictionary_v2.xlsx')

#And turn it into a dict with keys = old names and values = new names
data_q_lookup = {x:y for x,y in zip(data_dict.q,data_dict.q_name_short)}

#data_q_lookup

data_dict.loc[['Q14' in x for x in data_dict.q]]

,q,q_name_long,q_name_short,q.1,type,role,category
78,Q14_3,barriers- Limited opportunities to access finance,q14_barriers_finance,q14_barriers_,finance,NaN,NaN
79,Q14_4,barriers- Overall economic climate,q14_barriers_economic_climate,q14_barriers_,economic_climate,NaN,2.0
80,Q14_1,barriers- It is hard to recruit talent with th...,q14_barriers_skills_shortages,q14_barriers_,skills_shortages,NaN,2.0
81,Q14_2,barriers- Lack of supply of highly skilled wor...,q14_barriers_skills_shortages_education,q14_barriers_,skills_shortages_education,NaN,2.0
82,Q14_9,barriers- Unaware of latest research and devel...,q14_barriers_access_university_knowledge,q14_barriers_,access_university_knowledge,NaN,2.0
83,Q14_7,barriers- Poor technical infrastructure (e.g. ...,q14_barriers_technical_infrastructure,q14_barriers_,technical_infrastructure,NaN,2.0
84,Q14_8,barriers- Lack of knowledge on latest market r...,q14_barriers_market_knowledge,q14_barriers_,market_knowledge,NaN,2.0
85,Q14_5,barriers- Lack of formal advice and mentoring ...,q14_barriers_lack_advice,q14_barriers_,lack_advice,NaN,2.0
86,Q14_6,barriers- Limited supply of appropriate proper...,q14_barriers_office_space,q14_barriers_,office_space,NaN,2.0
87,Q14_11,"barriers- Government policy (e.g. tax breaks, ...",q14_barriers_government_policy,q14_barriers_,government_policy,NaN,2.0


### MTM survey

In [36]:
def load_clean_survey(survey_path,data_dict):
    '''
    This function loads a survey csv file and relabels the columns with a cleaned up data dict
    The data dict is a csv where the messy question names are mapped vs clean ones
    
    '''
    
    survey_messy = pd.read_csv(ext_data+survey_path,encoding='latin1')

    #We only keep the questions in the data dictionary
    survey_subset = survey_messy.loc[:,[x for x in survey_messy.columns if x in list(data_dict.keys())]]

    #We rename the messy names based on the lookup
    survey_subset.columns= [data_dict[x] for x in survey_subset.columns]

    return(survey_subset)

In [37]:
mtm_from_spss_v1 = load_clean_survey('/14_2_2018_immersive_survey.csv',data_q_lookup)


#Get the part of this which has company metadata
meta_vars = ['id','orgname','q2_3_postcode','q2_4_email']

In [38]:
#Load the second survey (which has been cleaned and for some reason doesn't include the metadata)

mtm_from_spss_v2 = load_clean_survey(survey_path='/14_2_2018_immersive_survey_v2.csv',data_dict=data_q_lookup) 

In [39]:
mtm_merged = pd.merge(mtm_from_spss_v1[meta_vars],mtm_from_spss_v2,left_on='id',right_on='id')

In [40]:
#Clean postcodes
mtm_merged['q2_3_postcode'] = [x.upper().strip() for x in mtm_merged['q2_3_postcode']]

mtm_merged['clean_postcode'] = [x if ' ' in x else x[:2]+' '+x[2:] for x in mtm_merged['q2_3_postcode']]

In [41]:
#Add geocodes

mtm_geo = pd.merge(mtm_merged,nspl[['pcds','ttwa','gor']],left_on='clean_postcode',right_on='pcds',how='left')
mtm_geo['ttwa_name']= [ttwa_codes_names_lookup[x] if x in ttwa_codes_names_lookup.keys() else np.nan for x in mtm_geo['ttwa']]
mtm_geo['reg_nat'] = [gor_names_lookup[x] if x in gor_names_lookup.keys() else np.nan for x in mtm_geo['gor']]

In [42]:
mtm_geo['reg_nat'].value_counts()

London                      84
South East                  37
North West                  30
South West                  21
Yorkshire and The Humber    15
Scotland                    13
Northern Ireland            11
East of England             10
Wales                        7
West Midlands                5
East Midlands                5
North East                   3
Name: reg_nat, dtype: int64

In [43]:
#Let us extract the web domain of those companies where we have failed to match the postcodes
mtm_geo['domain'] = [x.split('@')[1].strip().lower() for x in mtm_geo['q2_4_email']]


mtm_no_ttwa_domain = [x for x in mtm_geo.loc[mtm_geo.ttwa_name.isna(),'domain']]

In [44]:
#Some quick checks

mtm_geo.shape

(278, 148)

278 responses

In [45]:
#Are there any repeated domains?
domain_counts = mtm_geo.loc[[x !='gmail.com' for x in mtm_geo.domain],'domain'].value_counts()

#Which ones?
repeated_domains = domain_counts.index[domain_counts>1]

In [46]:
#How many?
print(mtm_geo.loc[[x !='gmail.com' for x in mtm_geo.domain],:].duplicated('domain').sum())

5


5 duplicated domains

In [47]:
#What are the responses?
mtm_geo.loc[[x in repeated_domains for x in mtm_geo.domain],:].sort_values('domain')

,id,orgname,q2_3_postcode,q2_4_email,q3_market_role_vr_techdev,q3_market_role_ar_techdev,q3_market_role_other_ie_techdev,q3_market_role_vr_content,q3_market_role_ar_content,q3_market_role_other_ie_content,...,q3_role_network,q3_role_finance,q3_role_other,clean_postcode,pcds,ttwa,gor,ttwa_name,reg_nat,domain
7,3350337,Nuclear AMRC ...,S60 5WG,r.w.scott@amrc.co.uk ...,Yes,Yes,No,Yes,Yes,No,...,No,No,No,S60 5WG,S60 5WG,E30000261,E12000003,Sheffield,Yorkshire and The Humber,amrc.co.uk
99,3373758,AMRC with Boeing ...,S9 1ZA,c.freeman@amrc.co.uk ...,Yes,Yes,Yes,Yes,Yes,Yes,...,"Trade body, association or industry network",No,No,S9 1ZA,S9 1ZA,E30000261,E12000003,Sheffield,Yorkshire and The Humber,amrc.co.uk
173,3392856,BBC Research & Development ...,M50 2LH,simon.lumb@bbc.co.uk ...,Yes,Yes,No,Yes,Yes,No,...,No,No,No,M50 2LH,M50 2LH,E30000239,E12000002,Manchester,North West,bbc.co.uk
270,3627440,BBC Cymru Wales ...,CF5 2YQ,robin.moore@bbc.co.uk ...,No,No,No,Yes,Yes,Yes,...,No,No,No,CF5 2YQ,CF5 2YQ,W22000024,W99999999,Cardiff,Wales,bbc.co.uk
39,3351034,South Down Project ...,BN44 3TX,robinbrownsell@me.com ...,No,No,No,Yes,Yes,No,...,No,No,No,BN44 3TX,BN44 3TX,E30000292,E12000008,Worthing,South East,me.com
274,3634452,Juice Immersive ...,L19 7NN,owencotterell@me.com ...,Yes,Yes,No,Yes,Yes,No,...,No,No,No,L19 7NN,L19 7NN,E30000233,E12000002,Liverpool,North West,me.com
62,3372580,Edinburgh Napier University ...,EH10 5DT,callum.egan@napier.ac.uk ...,Yes,Yes,Yes,No,No,No,...,"Trade body, association or industry network",No,No,EH10 5DT,EH10 5DT,S22000059,S99999999,Edinburgh,Scotland,napier.ac.uk
268,3626326,The Walt Disney Company and Edinburgh Napier U...,EH9 8AA,k.mitchell2@napier.ac.uk ...,Yes,Yes,Yes,Yes,Yes,Yes,...,No,Financial support / investment / public funder,Other,EH9 8AA,EH9 8AA,S22000059,S99999999,Edinburgh,Scotland,napier.ac.uk
12,3350348,The Old Market ...,BN3 1AS,james@theoldmarket.com ...,No,No,No,Yes,Yes,No,...,No,No,Other,BN3 1AS,BN3 1AS,E30000179,E12000008,Brighton,South East,theoldmarket.com
258,3404551,The Old Market ...,BN3 1AS,james@theoldmarket.com ...,No,No,No,Yes,Yes,No,...,"Trade body, association or industry network",Financial support / investment / public funder,No,BN3 1AS,BN3 1AS,E30000179,E12000008,Brighton,South East,theoldmarket.com


In [48]:
#Not that many 

mtm_geo = mtm_geo.drop_duplicates('q2_4_email')
mtm_geo.reset_index(drop=True,inplace=True)

np.sum(mtm_geo['ttwa_name'].isna())

37

### GlassAI data for some final checking

In [49]:
glass_ai = pd.read_excel(ext_data+'/17_11_2017_company_list_glassai.xlsx')
glass_ai.head()

glass_ai.shape

#Postcodes
glass_postcodes = pd.read_table(ext_data+'/immerse_v4_postcodes.txt',delimiter='\t',header=None)
glass_postcodes.columns=['id','postcode']

glass_ai_geo = pd.merge(glass_ai,glass_postcodes,left_on='Organization ID',right_on='id')
glass_ai_geo['postcode_top'] = [x.split(",")[0].upper() if type(x)==str else np.nan for x in glass_ai_geo.postcode]

glass_ai_geo = pd.merge(glass_ai_geo,nspl[['pcds','ttwa','lat','long','gor']],left_on='postcode_top',right_on='pcds',how='left')
glass_ai_geo['ttwa_name'] = [ttwa_codes_names_lookup[x] if x in ttwa_codes_names_lookup.keys() else np.nan for x in glass_ai_geo['ttwa']]
glass_ai_geo['reg_name'] = [gor_names_lookup[x] if x in gor_names_lookup.keys() else np.nan for x in glass_ai_geo['gor']]

In [50]:
#This is the domain of the GlassAI companies
glass_ai_geo['domain'] = [re.sub('www.','',x) for x in glass_ai['Website']]

### Fill missing MTM geo data with the ttwa / region data we found in GlassAI

In [51]:
#What's the overlap?
len(set(mtm_no_ttwa_domain) & set(glass_ai_geo.domain))
#An extra 18 companies


#We need to find a way of assigning them the GlassAI regions and names
glass_ai_geo_data = glass_ai_geo.loc[[x in set(mtm_no_ttwa_domain) & set(glass_ai_geo.domain) for x in glass_ai_geo.domain],
                 ['domain','gor','ttwa_name','reg_name','lat','long']]

#This dict takes a glass ai domain and returns its geo stuff. 
glass_ai_geo_dict = glass_ai_geo_data.to_dict(orient='records')
glass_ai_geo_dict = {x['domain']:x for x in glass_ai_geo_dict}

In [52]:
#We loop over variables. If their domains are missing in MTM but have data in GlassAI we assign them the
#GlassAI geo info

for x in np.arange(len(mtm_geo)):
    
    dom = str(mtm_geo.loc[x,'domain'])
    
    #If the domain is in the overlap between glass ai and mtm
    
    if dom in set(mtm_no_ttwa_domain) & set(glass_ai_geo.domain):
        mtm_geo.loc[x,'ttwa_name']=glass_ai_geo_dict[dom]['ttwa_name']
        mtm_geo.loc[x,'reg_nat']=glass_ai_geo_dict[dom]['reg_name']


In [56]:
np.sum(mtm_geo['ttwa_name'].isna())

31

In [55]:
#Pickle all the processed data

with open(proc_data+'/{date}_immersive_dataset.p'.format(date=today_str),'wb') as outfile:
    pickle.dump([glass_ai_geo,mtm_geo],outfile)

In [23]:
check.loc[:,['Q10_' in x for x in check.columns]].dropna(
).applymap(lambda x: 1 if 'Very' in x else 0).sum().sort_values(ascending=False)

NameError: name 'check' is not defined

In [ ]:
check = pd.read_csv(ext_data+'/14_2_2018_immersive_survey_v2.csv')

In [57]:
today_str

'4_3_2018'